<a href="https://colab.research.google.com/github/mateusribeirocampos/diollm/blob/main/YOLOv3_Dec_seg_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Criação do diretório darknet para o modelo yolo a partir do repositório de Alexey AB.

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!make clean && make

## Instalação da pacote lipopencv-dv e tree

 It's the development version of OpenCV (Open Source Computer Vision Library), a popular library used for computer vision tasks (like image processing, object detection, etc.). The development version includes header files and libraries needed to build applications that use OpenCV.

 o pacote tree facilita a visualização da estrutura do projeto

In [ ]:
!apt-get install libopencv-dev
!sudo apt-get install tree

## Criação dos arquivos

Os arquivos vão receber as configurações e os database para o treino, validação e testes.

In [ ]:
!mkdir -p /content/backup
!mkdir -p /content/coco/images
!mkdir -p /content/coco/annotations

## Fazer o download

Aqui é feito o download do database todos compactados

In [ ]:
# Baixar val2017 (1GB)
!wget -c http://images.cocodataset.org/zips/val2017.zip -P /content/coco/images

# Baixar test2017
!wget -c http://images.cocodataset.org/zips/test2017.zip -P /content/coco/images

# Baixar anotações (train + val)
!wget -c http://images.cocodataset.org/annotations_trainval2017.zip -P /content/coco/annotations

# Baixar imagens de treino (18GB)
!wget -c http://images.cocodataset.org/zips/train2017.zip -P /content/coco/images

## Desconpactação do database

In [ ]:
# Descompactar validações
!unzip /content/coco/images/val2017.zip -d /content/coco/images
!rm /content/coco/images/val2017.zip

# Descompactar imagens de testes
!unzip /content/coco/images/test2017.zip -d /content/coco/images
!rm /content/coco/images/test2017.zip

In [ ]:
# Descompactar imagens
!unzip /content/coco/images/train2017.zip -d /content/coco/images
!rm /content/coco/images/train2017.zip

# Descompactar anotações de treino e validações
!unzip /content/coco/annotations/annotations_trainval2017.zip -d /content/coco/annotations
!rm /content/coco/annotations/annotations_trainval2017.zip

## Verificação dos arquivos annotations

In [ ]:
!ls /content/coco/annotations/

## Estrutura dos diretórios

In [ ]:
!tree -d /content/coco/

In [ ]:
!tree -d /content/coco/images/

# Função de criação dos arquvivos para treino e validação.

In [ ]:
import os
import shutil
from pycocotools.coco import COCO

def setup_coco_directories(base_dir: str, splits: list):
    """Cria a estrutura de diretórios para o dataset filtrado"""
    os.makedirs(base_dir, exist_ok=True)
    for split in splits:
        os.makedirs(os.path.join(base_dir, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(base_dir, split, 'labels'), exist_ok=True)

def create_config_files(base_dir: str, categories: list):
    """Cria os arquivos de configuração obj.names e obj.data"""
    # obj.names
    with open(os.path.join(base_dir, 'obj.names'), 'w') as f:
        f.write('\n'.join(categories))

    # obj.data
    config_content = f"""classes = {len(categories)}
    train = {os.path.join(base_dir, 'train.txt')}
    valid = {os.path.join(base_dir, 'val.txt')}
    names = {os.path.join(base_dir, 'obj.names')}
    backup = /content/backup
    """
    with open(os.path.join(base_dir, 'obj.data'), 'w') as f:
        f.write(config_content)

def generate_split_filelists(base_dir: str, splits: list):
    """Gera os arquivos train.txt e val.txt com caminhos absolutos"""
    for split in splits:
        if split == 'test': continue  # Test não tem anotações

        img_dir = os.path.join(base_dir, split, 'images')
        output_file = os.path.join(base_dir, f'{split}.txt')

        with open(output_file, 'w') as f:
            for img_name in os.listdir(img_dir):
                f.write(f"{img_dir}/{img_name}\n")

def process_coco_split(splits: list, categories: list, max_samples_per_split: int = None):
    """
    Processa splits do COCO (train, val, test), filtrando por categorias e convertendo para YOLO.
    """
    # Criar diretório principal
    os.makedirs('/content/coco/filtered/', exist_ok=True)

    # Criar arquivo de classes
    with open('/content/coco/filtered/obj.names', 'w') as f:
        for cat in categories:
            f.write(cat + '\n')

    for split in splits:
        print(f"\nProcessando split: {split}")

        # Caminhos das imagens
        img_source_dir = f'/content/coco/images/{split}2017'
        img_dest_dir = f'/content/coco/filtered/{split}/images'
        os.makedirs(img_dest_dir, exist_ok=True)

        # Processar apenas train/val (test não tem anotações)
        if split != 'test':
            # Carregar anotações
            coco = COCO(f'/content/coco/annotations/instances_{split}2017.json')
            cat_ids = coco.getCatIds(catNms=categories)
            img_ids = coco.getImgIds(catIds=cat_ids)

            # Limitar amostras se especificado
            if max_samples_per_split:
                img_ids = img_ids[:max_samples_per_split]

            images = coco.loadImgs(img_ids)

            # Copiar imagens
            for img in images:
                src = os.path.join(img_source_dir, img['file_name'])
                dst = os.path.join(img_dest_dir, img['file_name'])
                shutil.copy(src, dst)

            # Criar diretório de labels
            ann_dest_dir = f'/content/coco/filtered/{split}/labels'
            os.makedirs(ann_dest_dir, exist_ok=True)

            # Converter anotações para YOLO
            for img in images:
                ann_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids)
                annotations = coco.loadAnns(ann_ids)

                label_path = os.path.join(ann_dest_dir, img['file_name'].replace('.jpg', '.txt'))
                with open(label_path, 'w') as f:
                    for ann in annotations:
                        x, y, w, h = ann['bbox']
                        img_w, img_h = img['width'], img['height']

                        cx = (x + w/2) / img_w
                        cy = (y + h/2) / img_h
                        nw = w / img_w
                        nh = h / img_h

                        class_id = categories.index(coco.loadCats(ann['category_id'])[0]['name'])
                        f.write(f"{class_id} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}\n")

        # Processar test (sem anotações)
        else:
            # Listar todas as imagens do diretório test2017
            images = [{'file_name': f} for f in os.listdir(img_source_dir)]
            if max_samples_per_split:
                images = images[:max_samples_per_split]

            # Copiar imagens
            for img in images:
                src = os.path.join(img_source_dir, img['file_name'])
                dst = os.path.join(img_dest_dir, img['file_name'])
                shutil.copy(src, dst)

# Configuração centralizada
SPLITS = ['train', 'val', 'test']
CATEGORIES = ['boat', 'person']
MAX_SAMPLES = 100  # None para usar todos

process_coco_split(
    splits=SPLITS,
    categories=CATEGORIES,
    max_samples_per_split=MAX_SAMPLES
)

## Verificação dos diretórios de treino e validação

In [ ]:
# Verifique se existem e contêm caminhos das imagens
!ls /content/coco/filtered/train.txt /content/coco/filtered/val.txt

# Exemplo do conteúdo de train.txt (primeiras 2 linhas)
!head -n 2 /content/coco/filtered/train.txt

In [ ]:
!tree /content/coco/filtered -L 2

## Verificação de Integridade

In [ ]:
def validate_dataset(base_dir: str, splits: list):
    """Valida se número de imagens e labels corresponde"""
    for split in splits:
        img_dir = os.path.join(base_dir, split, 'images')
        label_dir = os.path.join(base_dir, split, 'labels'))

        num_images = len(os.listdir(img_dir))
        num_labels = len(os.listdir(label_dir)) if split != 'test' else 0

        print(f"{split}: {num_images} imagens, {num_labels} labels")
        if split != 'test' and num_images != num_labels:
            raise ValueError("Inconsistência entre imagens e labels!")

## Controle de Versão dos Arquivos Gerados

In [ ]:
def save_dataset_version(base_dir: str):
    """Salva metadados da versão do dataset"""
    version_info = {
        'created_at': datetime.now().isoformat(),
        'categories': CATEGORIES,
        'splits': SPLITS,
        'num_samples': {split: len(os.listdir(os.path.join(base_dir, split, 'images')))
                       for split in SPLITS}
    }

    with open(os.path.join(base_dir, 'version.json'), 'w') as f:
        json.dump(version_info, f, indent=2)

## Documentação

In [ ]:
def generate_documentation(base_dir: str):
    """Cria arquivo README com informações do dataset"""
    readme_content = f"""# COCO Filtered Dataset

    ## Categories
    {', '.join(CATEGORIES)}

    ## Splits
    {', '.join(SPLITS)}

    ## Statistics
    """
    for split in SPLITS:
        num_images = len(os.listdir(os.path.join(base_dir, split, 'images')))
        readme_content += f"\n- {split}: {num_images} images"

    with open(os.path.join(base_dir, 'README.md'), 'w') as f:
        f.write(readme_content)

## Adição dos pesos para treino

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

## Verificação do diretório onde os pesos foram salvos

In [ ]:
!which yolov3.weights

## Verificação dos nomes

In [ ]:
!cat /content/coco/filtered/obj.names

boat
person


## Verificação dos dados

In [ ]:
!cat /content/coco/filtered/obj.data

classes = 2
train = /content/coco/filtered/train.txt
valid = /content/coco/filtered/val.txt
names = /content/coco/filtered/obj.names
backup = /content/backup


## Configuração dos arquvios para treino

In [ ]:
# Cria cópia do cfg original
!cp cfg/yolov3.cfg cfg/yolov3_custom.cfg

# Aplica modificações
!sed -i 's/batch=1/batch=64/' cfg/yolov3_custom.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_custom.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_custom.cfg
!sed -i 's/classes=80/classes=2/' cfg/yolov3_custom.cfg
!sed -i 's/filters=255/filters=21/' cfg/yolov3_custom.cfg

## Execução dos testes

In [ ]:
!./darknet detector train /content/coco/filtered/obj.data cfg/yolov3_custom.cfg yolov3.weights -dont_show -map

In [ ]:
!tree /content/coco/filtered/train -L 2